# Cassandra University application

##Step 1: Setup Cassandra Cluster on Docker

We will install Cassandra on Docker; we will have a full description on Containers and Dockers later in the course.

Make sure Docker Desktop is installed. If not, download and install Docker from the official website [Get Docker | Docker Docs](https://docs.docker.com/get-docker/).

In [ ]:
# On the first CMD:
# Pull the Cassandra Docker Image:
docker pull cassandra:latest

In [ ]:
# Start the first Cassandra node (Seed Node):
docker run --name cass_cluster cassandra:latest

In [ ]:
# Once the cluster is running, open a new cmd and run the command:
docker exec -it cass_cluster cqlsh
# You can interact now with the cluster using CQL.

##Step 2: Create University Keyspace

In [ ]:
CREATE KEYSPACE IF NOT EXISTS University WITH replication = {
    'class': 'SimpleStrategy',
    'replication_factor’: 1
};

In [ ]:
# To validate you created the keyspace
DESCRIBE keyspaces;

In [ ]:
# The output should be:
system       system_distributed  system_traces  system_virtual_schema
system_auth  system_schema       system_views   university

## Step 3: Creating Tables

In [ ]:
USE university;

In [ ]:
CREATE TABLE students (
  student_id UUID PRIMARY KEY,
  name TEXT,
  email TEXT,
  enrollment_year INT
);

In [ ]:
CREATE TABLE courses (
  course_id UUID PRIMARY KEY,
  course_name TEXT,
  lecturer_id UUID,
  credits INT
);

In [ ]:
CREATE TABLE lecturers (
  lecturer_id UUID PRIMARY KEY,
  name TEXT,
  department TEXT
);

In [ ]:
CREATE TABLE course_enrollments (
  course_id UUID,
  student_id UUID,
  enrollment_date DATE,
  PRIMARY KEY (course_id, student_id)
);

In [ ]:
# To validate you created the tables
cqlsh:university> DESCRIBE tables;

In [ ]:
# The output should be:
course_enrollments  courses  lecturers  students

## Step 4: Inserting data

In [ ]:
# Inserting data into students
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (550e8400-e29b-41d4-a716-446655440000, 'John Doe', 'john.doe@email.com', 2021);
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (652e8500-f39c-42d5-b517-557655450001, 'Jane Smith', 'jane.smith@email.com', 2020);
INSERT INTO students (student_id, name, enrollment_year) VALUES (753e8600-e49d-53e6-c628-668655460002, 'Alice Johnson', 2019);

In [ ]:
# Inserting data into lecturers
INSERT INTO lecturers (lecturer_id, name, department) VALUES (860e9700-a59e-63f7-d739-779655470003, 'Dr. James Watson', 'Biology');
INSERT INTO lecturers (lecturer_id, name, department) VALUES (961e9800-b69f-73c8-e840-889655480004, 'Dr. Emma Wilson', 'Physics');
INSERT INTO lecturers (lecturer_id, name) VALUES (072e9900-d710-83e9-f951-990655490005, 'Prof. Michael Brown');

In [ ]:
# Inserting data into courses
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (183fa000-f821-93a0-b062-a96655650006, 860e9700-a59e-63f7-d739-779655470003, 'Biology 101', 3);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (284fb000-a932-a4b1-c173-b97755660007, 961e9800-b69f-73c8-e840-889655480004, 'Physics 101', 4);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (385fc000-d043-b5e2-f284-c98855670008, 072e9900-d710-83e9-f951-990655490005, 'Mathematics 101', 3);

In [ ]:
# Inserting data into course_enrollments
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (183fa000-f821-93a0-b062-a96655650006, 550e8400-e29b-41d4-a716-446655440000, '2023-10-01');
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (284fb000-a932-a4b1-c173-b97755660007, 652e8500-f39c-42d5-b517-557655450001, '2023-09-15');
INSERT INTO course_enrollments (course_id, student_id) VALUES (385fc000-d043-b5e2-f284-c98855670008, 753e8600-e49d-53e6-c628-668655460002);

## Step 5: Querying the data

In [1]:
# Get all students
SELECT * FROM STUDENTS

>>
 student_id                           | email                | enrollment_year | name
--------------------------------------+----------------------+-----------------+---------------
 753e8600-e49d-53e6-c628-668655460002 |                 null |            2019 | Alice Johnson
 652e8500-f39c-42d5-b517-557655450001 | jane.smith@email.com |            2020 |    Jane Smith
 550e8400-e29b-41d4-a716-446655440000 |   john.doe@email.com |            2021 |      John Doe
 (3 rows)

In [ ]:
# Get the list of students who enrolled after 2020 along with their email and the courses they enrolled in
SELECT student_id, name, email
FROM students
WHERE enrollment_year > 2020 ALLOW FILTERING;

>>
 student_id                           | name     | email
--------------------------------------+----------+--------------------
 550e8400-e29b-41d4-a716-446655440000 | John Doe | john.doe@email.com

(1 rows)

In [ ]:
# Find all courses and count of students enrolled in them
SELECT course_id, COUNT(student_id) as student_count
FROM course_enrollments
GROUP BY course_id;

>>
 course_id                            | student_count
--------------------------------------+---------------
 183fa000-f821-93a0-b062-a96655650006 |             1
 385fc000-d043-b5e2-f284-c98855670008 |             1
 284fb000-a932-a4b1-c173-b97755660007 |             1

(3 rows)